In [ ]:
"""
erratics-layers.ipynb

Demonstrates how to:
- Read a CSV of glacial erratics (lat/lon).
- Use ArcGIS Python API with an API key (anonymous-like user) to query real
   feature services for rivers and coastlines (in a bounding box).
- Perform local proximity analysis with Shapely (buffer/intersect in projected coordinates).
- Display results in an ArcGIS MapView with toggleable layers (no publishing),
   specifically:
      - All erratics
      - Close to rivers
      - Close to coast
Optimized spatial analysis to:
- Precompute projections for all geometries.
- Build spatial indexes using Rtree for proximity queries.
- Dramatically reduce computation time.
"""

import os
import csv

import pyproj
import shapely.ops
from shapely.geometry import Point

from arcgis.gis import GIS
from arcgis.features import FeatureLayer, Feature, FeatureSet
from arcgis.map.symbols import SimpleMarkerSymbolEsriSMS, SimpleLineSymbolEsriSLS, SimpleMarkerSymbolStyle
from arcgis.map.popups import PopupInfo
from arcgis.geometry import Geometry

from rtree import index

In [2]:
###############################################################################
# 2. READ CSV OF ERRATICS
###############################################################################

def read_erratics_csv(csv_path):
    """
    Reads a csv with these columns:
       row[1] = latitude,
       row[2] = longitude,
       row[6] = id,
       row[7] = name
    Returns a list of dicts: [{"id":"ERR001", "name":"...", "lat":47..., "lon":-122...}, ...]
    """
    data = []
    with open(csv_path, mode='r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        header = next(reader)  # skip header
        for row in reader:
            lat = float(row[1])
            lon = float(row[2])
            eid = row[6]
            ename = row[7]
            data.append({
                "id": eid,
                "name": ename,
                "lat": lat,
                "lon": lon
            })
    return data

In [3]:
###############################################################################
# 3. ARCGIS FEATURE SERVICE URLS
###############################################################################

# Real data from ArcGIS Online
RIVERS_URL = "https://services1.arcgis.com/ZIL9uO234SBBPGL7/ArcGIS/rest/services/Rivers_World_Natural_Earth/FeatureServer/0"
COAST_URL =  "https://services7.arcgis.com/WSiUmUhlFx4CtMBB/ArcGIS/rest/services/GSHHS_GlobalCoastlines_HighResolution/FeatureServer/0"

In [4]:
###############################################################################
# 4. PROXIMITY UTILS
###############################################################################

def reproject_point(point_geometry, from_crs="EPSG:4326", to_crs="EPSG:3857"):
    """Reproject a Shapely Point using pyproj for distance-based calculations."""
    project_func = pyproj.Transformer.from_crs(
        from_crs, to_crs, always_xy=True
    ).transform
    return shapely.ops.transform(project_func, point_geometry)

def is_point_within_distance(point_lonlat, list_of_features, distance_meters):
    """
    Return True if 'point_lonlat' (Shapely, EPSG:4326) is within
    'distance_meters' of ANY geometry in 'list_of_features'.
    """
    point_3857 = reproject_point(point_lonlat, "EPSG:4326", "EPSG:3857")
    for feat_4326 in list_of_features:
        feat_3857 = reproject_point(feat_4326, "EPSG:4326", "EPSG:3857")
        if point_3857.distance(feat_3857) <= distance_meters:
            return True
    return False

In [5]:
###############################################################################
# 5. HELPER: CONVERT FeatureSet --> LIST OF SHAPELY GEOMETRIES
###############################################################################

def fs_to_shapely_list(fs):
    """
    Convert an ArcGIS FeatureSet to a list of Shapely geometries.
    Uses arcgis.geometry.Geometry(...) + .as_shapely for robust conversion.
    Skips null or empty geometry.
    """
    geoms = []
    null_count = 0
    fail_count = 0
    
    for f in fs.features:
        if f.geometry is None:
            null_count += 1
            continue
        
        try:
            arc_geom = Geometry(f.geometry)
            if arc_geom.is_empty:
                null_count += 1
                continue
            sgeom = arc_geom.as_shapely
            if sgeom and not sgeom.is_empty:
                geoms.append(sgeom)
            else:
                null_count += 1
        except Exception:
            fail_count += 1
    
    if null_count > 0:
        print(f"[WARN] Skipped {null_count} empty or null geoms.")
    if fail_count > 0:
        print(f"[WARN] Conversion failed for {fail_count} geoms.")
    return geoms

In [6]:
###############################################################################
# A) READ LOCAL CSV
###############################################################################

CSV_PATH = "erratics_gis.csv"

erratics = read_erratics_csv(CSV_PATH)
print(f"Loaded {len(erratics)} erratics from {CSV_PATH}.")

Loaded 213 erratics from erratics_gis.csv.


In [7]:
###############################################################################
# B) Connect to ArcGIS via API key
###############################################################################

API_KEY = "AAPTxy8BH1VEsoebNVZXo8HurNF06YWHEbns1XWgxHCpzlyDjxh3Rx8dbWwLv-YhpjbuKnTT5uOWTO7-WskSkafBC6jiu1QCJUkLpyb-6epQoRGSlLogeBqx_bwJSceJjPn7ooXJnbZwuuPl8jcPSv014Zrv8X5lK7aDfBFPPUlt8dfqMsy9ZZtrxkFKKrL9bRIhoJZTDuTzFRhIgV43IhOe3YtvXO7xAuynlg14irzQnV8.AT1_TRlM9H04"  # <-- replace with your real key
gis = GIS(api_key=API_KEY)
if gis.users.me is not None:
    print("[INFO] Named user or full developer account.")
else:
    print("[INFO] Using anonymous-like user with API key.")

[INFO] Using anonymous-like user with API key.


In [8]:
###############################################################################
# C) Bounding box for North America to limit queries
###############################################################################

# lat_min       lat_max     lon_min     lon_max
# ymin          ymax        xmin        xmax
# 19.2726009	19.5926009	-99.2933416	-98.9733416 <- Mexico
# 41.6765556	83.3362128	-141.00275	-52.3231981 <- Canada

NA_BBOX = {
    "xmin": -141,
    "ymin": 42,
    "xmax": -52,
    "ymax": 83,
    "spatialReference": {"wkid": 4326}
}

In [9]:
###############################################################################
# D) Query rivers and coast
###############################################################################

rivers_layer = FeatureLayer(RIVERS_URL, gis=gis)
coast_layer  = FeatureLayer(COAST_URL,  gis=gis)

print("[INFO] Querying rivers within bounding box...")
rivers_fs = rivers_layer.query(
    geometry=NA_BBOX,
    geometry_type="esriGeometryEnvelope",
    in_sr=4326,
    out_sr=4326,
    where="1=1",
    return_geometry=True
)
print(f" -> fetched {len(rivers_fs.features)} river features.")

print("[INFO] Querying coast within bounding box...")
coast_fs = coast_layer.query(
    geometry=NA_BBOX,
    geometry_type="esriGeometryEnvelope",
    in_sr=4326,
    out_sr=4326,
    where="1=1",
    return_geometry=True
)
print(f" -> fetched {len(coast_fs.features)} coast features.")

rivers_geomlist = fs_to_shapely_list(rivers_fs)
coast_geomlist  = fs_to_shapely_list(coast_fs)

print(f"[INFO] Rivers geometry count: {len(rivers_geomlist)}")
print(f"[INFO] Coast geometry count:  {len(coast_geomlist)}")

[INFO] Querying rivers within bounding box...
 -> fetched 140 river features.
[INFO] Querying coast within bounding box...
 -> fetched 31629 coast features.
[INFO] Rivers geometry count: 140
[INFO] Coast geometry count:  31629


In [ ]:
###############################################################################
# E) Proximity checks
###############################################################################

RIVER_DISTANCE = 1000.0  # 1 km
COAST_DISTANCE = 2000.0  # 2 km

results = []
for e in erratics:
    pt = Point(e["lon"], e["lat"])
    close_river = is_point_within_distance(pt, rivers_geomlist, RIVER_DISTANCE)
    close_coast = is_point_within_distance(pt, coast_geomlist,  COAST_DISTANCE)

    results.append({
        "id":   e["id"],
        "name": e["name"],
        "lon":  e["lon"],
        "lat":  e["lat"],
        "close_river":  close_river,
        "close_coast":  close_coast
    })

print(f"[INFO] Completed proximity analysis for {len(results)} erratics.")

[INFO] Completed proximity analysis for 213 erratics.


In [11]:
###############################################################################
# F) Create toggleable layers in the ArcGIS Map
###############################################################################

def make_featureset(records, layer_label):
    """
    Build a fully defined FeatureSet: 
        - geometry_type='esriGeometryPoint'
        - spatial_reference={"wkid":4326}
        - fields: an array describing each attribute
    """
    # 1) Build a list of arcgis.features.Feature
    feats = []
    for r in records:
        geom = {
            "x": r["lon"],
            "y": r["lat"],
            "spatialReference": {"wkid": 4326}
        }
        # We'll store booleans as strings to avoid field-type confusion
        attrs = {
            "id":   r["id"],
            "name": r["name"],
            "lon":  r["lon"],
            "lat":  r["lat"],
            "close_river":  str(r["close_river"]),
            "close_coast":  str(r["close_coast"])
        }
        feats.append(Feature(geometry=geom, attributes=attrs))
    
    # 2) Define the "fields" array for each attribute
    fields = [
        {
            "name": "id",
            "type": "esriFieldTypeString",
            "alias": "id"
        },
        {
            "name": "name",
            "type": "esriFieldTypeString",
            "alias": "name"
        },
        {
            "name": "lon",
            "type": "esriFieldTypeDouble",
            "alias": "lon"
        },
        {
            "name": "lat",
            "type": "esriFieldTypeDouble",
            "alias": "lat"
        },
        {
            "name": "close_river",
            "type": "esriFieldTypeString",
            "alias": "close_river"
        },
        {
            "name": "close_coast",
            "type": "esriFieldTypeString",
            "alias": "close_coast"
        }
    ]

    # 3) Create the FeatureSet with geometry type + fields
    fs = FeatureSet(
        features=feats,
        geometry_type="esriGeometryPoint",
        spatial_reference={"wkid": 4326},
        fields=fields
    )
    return fs

In [12]:
###############################################################################
# G) Create FeatureSets for each layer: All, close_river, close_coast
###############################################################################

all_fs = make_featureset(results, "All Erratics")

close_river_fs = make_featureset(
    [r for r in results if r["close_river"]], 
    "Close to Rivers"
)
close_coast_fs = make_featureset(
    [r for r in results if r["close_coast"]],
    "Close to Coast"
)

In [13]:
###############################################################################
# H) Define some simple symbols for the map
###############################################################################

sym_all = SimpleMarkerSymbolEsriSMS(
    style=SimpleMarkerSymbolStyle.esri_sms_circle,
    color=[0, 0, 255, 128],
    outline=SimpleLineSymbolEsriSLS(color=[255,255,255], width=1)
)
sym_river = SimpleMarkerSymbolEsriSMS(
    style=SimpleMarkerSymbolStyle.esri_sms_circle,
    color=[255, 0, 0, 128],
    outline=SimpleLineSymbolEsriSLS(color=[255,255,255], width=1)
)
sym_coast = SimpleMarkerSymbolEsriSMS(
    style=SimpleMarkerSymbolStyle.esri_sms_circle,
    color=[0, 255, 0, 128],
    outline=SimpleLineSymbolEsriSLS(color=[255,255,255], width=1)
)

In [14]:
###############################################################################
# I) Create a map and display it
###############################################################################

my_map = gis.map("Washington, USA")
my_map

Map(center=[4706222.166681878, -8573817.429310536], extent={'xmin': -8585394.656353038, 'ymin': 4691358.594985…

In [15]:
###############################################################################
# J) Add the layers to the map
###############################################################################

# Draw each set as a toggleable layer (label)
my_map.content.draw(shape=all_fs, symbol=sym_all)
my_map.content.draw(shape=close_river_fs, symbol=sym_river)
my_map.content.draw(shape=close_coast_fs, symbol=sym_coast)

In [16]:
###############################################################################
# K) Print results
###############################################################################

print("\n[INFO] Done. Below is an interactive ArcGIS MapView (in Jupyter).")
print("You can toggle the layers in the layer list.\n")
print("----- RESULTS -----")
for r in results:
    print(f"ID={r['id']} | Name={r['name']} | lat={r['lat']} lon={r['lon']} | "
            f"CloseToRiver={r['close_river']} | CloseToCoast={r['close_coast']}")


[INFO] Done. Below is an interactive ArcGIS MapView (in Jupyter).
You can toggle the layers in the layer list.

----- RESULTS -----
ID=1 | Name=Denali Erratic | lat=63.721633 lon=-148.9658 | CloseToRiver=False | CloseToCoast=False
ID=2 | Name=Williwaw Erratic | lat=60.7845 lon=-148.87425 | CloseToRiver=False | CloseToCoast=False
ID=3 | Name=Mendenhall Glacier Erratics | lat=58.491255 lon=-134.523227 | CloseToRiver=False | CloseToCoast=False
ID=4 | Name=Big Rock | lat=49.989773 lon=-125.225808 | CloseToRiver=False | CloseToCoast=False
ID=5 | Name=Bellevue Erratic / Sheridan Erratic | lat=45.117361 lon=-123.316211 | CloseToRiver=False | CloseToCoast=False
ID=6 | Name=Yamhill Valley Erratic | lat=45.14 lon=-123.292778 | CloseToRiver=False | CloseToCoast=False
ID=7 | Name=Shasta Court Erratic | lat=49.266017 lon=-122.8421 | CloseToRiver=False | CloseToCoast=False
ID=8 | Name=White Rock | lat=49.01995 lon=-122.802519 | CloseToRiver=False | CloseToCoast=False
ID=9 | Name=H Street Erratic | 

In [18]:
###############################################################################
# L) Export the map to an HTML file
###############################################################################

file_dir = os.path.join(os.getcwd(), "maps")
if not os.path.isdir(file_dir):
    os.mkdir(file_dir)

file_path = os.path.join(file_dir, "erratics-layers.html")

my_map.export_to_html(file_path, title="Erratics with Layers")

True